In [12]:
import numpy as np
from collections import Counter
from tensorflow import keras
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [13]:
def split(word):
    return [char for char in word]

In [14]:
file1 = open("./input.txt","r")
str = []
temp = file1.readline()
temp = file1.readline()

dict = ['X', 'T', '%', 'S', 'M', 'K', 'P','\n', 'L', 'Z', 'N']
temp1 = []

while temp:
    if temp[0] == 'X':
        temp1 = ''.join(temp1)
        str.append(temp1)
        temp1 = []
    if temp[0] not in dict:
        temp1.append(temp)
    temp = file1.readline()

In [15]:
dict = ['F','G','A','B','C','D','E','^','_','=','f','g','a','b','c','d','e','m']

for i in range(len(str)):
    for j in str[i]:
        if j not in dict:
            str[i] = str[i].replace(j, '')
    str[i] = split(str[i])

In [16]:
i = 0

while i<len(str):
    temp = []
    j = 0
    while j<len(str[i]):
        if str[i][j] in "^_=":
            str[i][j] += str[i][j+1]
            temp.append(str[i][j])
            j+=2
        elif j<len(str[i])-1 and (str[i][j+1] == 'm'):
            str[i][j] += str[i][j+1]
            temp.append(str[i][j])
            j+=2
        else:
            temp.append(str[i][j])
            j+=1
    
    str[i] = temp
    i+=1

In [17]:
vocab_to_int = {}
int_to_vocab = {}

k = 1
for i in range(len(str)):
    for j in range(len(str[i])):
        if str[i][j] not in vocab_to_int:
            vocab_to_int[str[i][j]] = k
            int_to_vocab[k] = str[i][j]
            k += 1

for i in range(len(str)):
    str[i] = [vocab_to_int[w] for w in str[i]]

int_to_vocab[0] = ' '

In [18]:
print(vocab_to_int)

{'G': 1, 'A': 2, 'B': 3, 'c': 4, 'd': 5, '=e': 6, 'f': 7, 'g': 8, 'E': 9, 'F': 10, 'D': 11, '=E': 12, '^F': 13, 'a': 14, 'b': 15, 'e': 16, '=f': 17, 'C': 18, '^f': 19, '=c': 20, '=B': 21, '=F': 22, '_B': 23, '=A': 24, '^c': 25, '^D': 26, '_e': 27, '^g': 28, '=g': 29, '=G': 30, '^G': 31, '^d': 32, '=d': 33, '^C': 34, '=C': 35, '_b': 36, 'em': 37, '_A': 38, 'dm': 39, '^A': 40, '_E': 41, 'm': 42, '^e': 43, '^B': 44, 'am': 45, '^a': 46, '=a': 47, 'fm': 48, '_d': 49, 'Em': 50, 'cm': 51, 'gm': 52, '=m': 53, 'bm': 54, '=_': 55, '_m': 56, '_a': 57, '_c': 58, '_f': 59}


In [19]:
seq_length = 5
X = []
Y = []

for i in range(len(str)):
    for j in range(0, len(str[i]) - seq_length):
        seq_in = str[i][j:j + seq_length]
        seq_out = str[i][j + seq_length]
        out = []
        for k in range(len(vocab_to_int)+1):
            if k == seq_out:
                out.append(1)
            else:
                out.append(0)
        X.append(seq_in)
        Y.append(out)

In [20]:
l = len(X)

X = np.array(X)
Y = np.array(Y)

X = np.reshape(X, (l, seq_length, 1))
X = X/len(vocab_to_int)

In [21]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_length, 1), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation = 'sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [13]:
model.summary()

print(len(X))

model.fit(X[:100000], Y[:100000], epochs = 100, batch_size = 128)

model.save('rnn_music_v4.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 256)            264192    
_________________________________________________________________
dropout (Dropout)            (None, 5, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 60)                15420     
Total params: 804,924
Trainable params: 804,924
Non-trainable params: 0
_________________________________________________________________
464880
Epoch 1/50
782/782 [==============================] - 25s 29ms/step - loss: 2.6523
Epoch 2/50
782/782 [====

In [24]:
f = open("GeneratedSamples.txt","a")
model.load_weights('./rnn_music_v4.h5')

for j in range(250):
    result = ''
    start = np.random.randint(100000, len(X)-1)
    pattern = X[start]
    print("Seed:")
    print(''.join([int_to_vocab[min(int(value*len(int_to_vocab)),59)] for value in pattern]), end = "")
    for i in range(0,50):
        x = np.reshape(pattern, (1, len(pattern), 1))
        prediction = model.predict(x, verbose=0)
        s = sum(prediction[0])
        prediction[0] = prediction[0]/s
        index = np.random.choice(range(len(vocab_to_int)+1), p=prediction[0])
        result += int_to_vocab[index]
        index = index/(len(int_to_vocab)+1)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
        
    print(result)
    result += '\n'
    f.write(result)

Seed:
fceAedFDgGAgFDGBBBDFDCCEA^Ccfg^f^fDdgBDCACcdfAFEebAcdBaegB
Seed:
BeGEFbeg=gDccEGgDDcFfcCAcAcf_Bdf^f^cggfc_BgAG^dEDEABefAGcefdc
Seed:
gedcBGGAEFgef^dBBGBFCeEFgfdd=e=e^fgFag=ffdgaDaFDAFcGAcgcceBB
Seed:
AceaafbgaceBgDabdcGBdfcBDDBBcdegaBDdDGdFFGFFF=ceD^Feg=e^fdc
Seed:
GFBBdebbde=fga^cgG^cc^fgfadcAFGFAAFAG^GAFedBABdceGAegbDgdffg
Seed:
GGGdgEBAdcfgFaFAAAGGcGFEFBAABcFE^GgF=CFF^FDDgDAFFEcdgfe^feG
Seed:
bgedcBdADEDDGAG^FcGAGBgfcBGEcBccEc^cgCFAcdebefaddBgcegbae
Seed:
BeGABcdf^fGA^GgDAdedBBcAeffbbdag=gABcgacGbGBggfdedBecegEaE
Seed:
AGFGAgCeeggadAFEDFBDCBFEEAfafBGEDDFCfBEBggefBdgagadAFGG
Seed:
EGABAcGcAAFAFfdFddcAGA^FAdFGdBefdgBgeBcbcBDECAEgFgeEgceG
Seed:
DFBcdDAFCdGE=EGAdd=eD=gGGBBGAAddfcefgAcfAGFAEDdAeedGdBBAcd
Seed:
fe^dfedfAc=efFGDcEcGBBdeaddDBfBe^FDG^Fff^f^fcaeDGdBcGBEffBGFG
Seed:
egBeeAA^ccB_BcDFDFcDFBDADFeFfFCfFefeg^f=fg^fAFGGfGABgfBFAgff
Seed:
gfedBAFAdefefcBGGdAdggBBgaa^gdDCBfFFAdADaFFGAcA=fffd^c=fgaD
Seed:
cBdefGG=fceCffadfbb^fbAEBGGEEg^fcccABFeegafe=fb^dagBABGAfgfe
Seed:
ffdeeged

_fabcacbaAgBbcEGBccgefBbceaBfcfBFeBfeBBeaFGEBcEGGgaCfeaa
Seed:
GAdAdABdAceGdecGAEECgg=fBddeefaD^fcecfBedfeggfBddGAcEecCa
Seed:
dcdedaDDc=BeCfcdf^fdBGABAFDEFDBdAFa^FdDBGgeFCCaDFFAGD^DeEgA
Seed:
BAGdcbABEGGAD^CDeE^fGAAFDFFEdDecDfDgFBFCdBeddeAdGcdDBFBcB
Seed:
c_abcaefAcAEGFcDEfGEfdcBFcdFEDgFGAddefefdgbbcAeBAAadccGD
Seed:
AFEEFDCCBGFgGdg=fbggBeDEg=eAaddcdcGE=GFGBdbGGGBBD^FafgffBAA
Seed:
fedefaAfBefdgge^daacGFbfdbEedddag=eabdaGBcddcBcGcBEeBEffb
Seed:
ffdeffa^cadBgfecGagggadBGcABFDG^FGGAAGdBGBFGgfdA^FdfDAdGBB
Seed:
BdefaA^cDbaBcf^faffe^cgccdffc=eAcAFDBAEDaG^cGdDaGcdDafedceec
Seed:
cAFGGEDccegBDbcfdcBEdefec^cFaEGdg^fggeeebeaAcgAbBGeAAgDeA
Seed:
GAG^FG=fdgBegafbaffbgAdedcAB=c^f^dBDfDeDgCdEcAABdfegedea^f^d^ga
Seed:
^cdefdFAFC^FBGDF=FgAGcgGeaEabfcggdcgBcAcBEcBEagcDEcBAABAdG
Seed:
aBedeefg^fg=fBceADedadga^gbGdcfAfgaecfafeccAEfagEDGFAGffef
Seed:
dcBABBcfeFgbbgeABdedBBgEFBDfgFcfdcAGBddd=eecbadBgcGFEGAe
Seed:
ABcde^dABBB^dGBFcGGADABdEFdDeDdadGABFdGGADDBcBDADEBBeGAAD
Seed:
AaAcAAAFCDad^cga=fge=fccA^F